In [21]:
import pytrends
from pytrends.request import TrendReq
import pandas as pd
from tqdm import tqdm

In [5]:
# query trend information for the following keywords
keyword_list = ['Sea Level','Temperatures','Carbon Dioxide','Global Warming']#, 'Weather']
country_list = ['AR', 'US', 'GB', 'DE', 'AU']

# build the connection
pytrend = TrendReq(hl='en-US', tz=360)

# Initialize an empty DataFrame to store the results
results_df = None

# get the trend information
for country in country_list:

    pytrend.build_payload(kw_list=keyword_list, timeframe = 'today 5-y', geo=country)
    temp_df = pytrend.interest_over_time()
    temp_df['country'] = country
    temp_df.reset_index(inplace=True)

    if results_df is None:
        results_df = temp_df
    else:
        results_df = pd.concat([results_df, temp_df], axis=0)

    print(temp_df.head())

        date  Sea Level  Temperatures  Carbon Dioxide  Global Warming  \
0 2018-10-28          0            25              48              23   
1 2018-11-04          0             0              49               0   
2 2018-11-11         41            41              23              48   
3 2018-11-18          0             0               0              32   
4 2018-11-25          0             0              27              53   

   isPartial country  
0      False      AR  
1      False      AR  
2      False      AR  
3      False      AR  
4      False      AR  
        date  Sea Level  Temperatures  Carbon Dioxide  Global Warming  \
0 2018-10-28         11            25              26              20   
1 2018-11-04         11            25              26              19   
2 2018-11-11         12            32              26              23   
3 2018-11-18          9            32              16              19   
4 2018-11-25         13            27              29     

# CONNECT to REDSHIFT

In [18]:
from sqlalchemy import create_engine
import psycopg2

from os.path import expanduser
home = expanduser("~")

with open(fr"{home}\creds\pwd_redshift.txt",'r') as f:
    pwd= f.read()

connection_params = dict(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
        , dbname='data-engineer-database'
        , user='juanmlacasa_coderhouse'
        , password=pwd
        , port='5439'
)

engine = create_engine(f'postgresql+psycopg2://{connection_params["user"]}:{connection_params["password"]}@{connection_params["host"]}:{connection_params["port"]}/{connection_params["dbname"]}')

In [19]:
for country_code in tqdm(country_list):
    results_df[results_df['country']==country_code].drop(['country', 'isPartial'], axis=1).to_sql(f'trends_{country_code}'.lower(), engine, schema='juanmlacasa_coderhouse', if_exists='replace', index=False)

C:\Users\juanm\AppData\Local\Temp\ipykernel_18520\770884221.py:2: UserWarning: The provided table name 'trends_AR' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  results_df[results_df['country']==country_code].drop(['country', 'isPartial'], axis=1).to_sql(f'trends_{country_code}', engine, schema='juanmlacasa_coderhouse', if_exists='replace', index=False)
C:\Users\juanm\AppData\Local\Temp\ipykernel_18520\770884221.py:2: UserWarning: The provided table name 'trends_US' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  results_df[results_df['country']==country_code].drop(['country', 'isPartial'], axis=1).to_sql(f'trends_{country_code}', engine, schema='juanmlacasa_coderhouse', if_exists='replace', index=False)
C:\Users\juanm\AppData\Local\Temp\ipykernel_18520\770884221.py:2: UserWarnin